In [1]:
import os
import sys
sys.path.insert(0,'/home/nitin.jain/ner_aleju')
from __future__ import absolute_import, division, print_function, unicode_literals
import argparse
import random
from model.datasets import load_windows, load_articles, generate_examples
import model.features as features
import dataset.config as cfg

In [2]:
feature_generators = features.create_features()
windows = load_windows(load_articles(cfg.ARTICLES_FILEPATH), cfg.WINDOW_SIZE,
                           feature_generators, only_labeled_windows=True)
print("Adding example windows (up to max %d)..." % (cfg.COUNT_WINDOWS_TRAIN))
examples = generate_examples(windows, nb_append=cfg.COUNT_WINDOWS_TRAIN,
                                 nb_skip=cfg.COUNT_WINDOWS_TEST, verbose=True)

Creating features...
Loading top N unigrams...
Loading all unigrams...
Loading person name unigrams...
Creating gazetteer...
Loading windows...
Adding example windows (up to max 20000)...


In [7]:
f = open('/home/nitin.jain/ner_aleju/dataset/outfeatures.txt', 'w')
for feature_values_lists, labels in examples:
    f.write(str(feature_values_lists + ["------"] + labels))
f.close()

In [15]:
#for feature_values_lists, labels in examples:
 #   print(feature_values_lists[0])

In [4]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)
for feature_values_lists, labels in examples:
    trainer.append(feature_values_lists, labels)
trainer.train("run2_sample45000_withunigram")

Generated 500 of max 20000 examples
Generated 1000 of max 20000 examples
Generated 1500 of max 20000 examples
Generated 2000 of max 20000 examples
Generated 2500 of max 20000 examples
Generated 3000 of max 20000 examples
Generated 3500 of max 20000 examples
Generated 4000 of max 20000 examples
Generated 4500 of max 20000 examples
Generated 5000 of max 20000 examples
Generated 5500 of max 20000 examples
Generated 6000 of max 20000 examples
Generated 6500 of max 20000 examples
Generated 7000 of max 20000 examples
Generated 7500 of max 20000 examples
Generated 8000 of max 20000 examples
Generated 8500 of max 20000 examples
Generated 9000 of max 20000 examples
Generated 9500 of max 20000 examples
Generated 10000 of max 20000 examples
Generated 10500 of max 20000 examples
Generated 11000 of max 20000 examples
Generated 11500 of max 20000 examples
Generated 12000 of max 20000 examples
Generated 12500 of max 20000 examples
Generated 13000 of max 20000 examples
Generated 13500 of max 20000 exa

In [27]:
trainer.get_params()
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [57]:
from itertools import chain
from sklearn.preprocessing import LabelBinarizer
lbin=LabelBinarizer()
y_true=[['PER','LOC','LOC','O'],['LOC','LOC','ORG','O']]
y_pred=[['PER','LOC','ORG','O'],['LOC','PER','ORG','O']]
list(chain.from_iterable(y_true))
lbin.fit_transform(list(chain.from_iterable(y_true)))
lbin.transform(list(chain.from_iterable(y_pred)))
#set(lbin.classes_)

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

In [17]:
print(cfg.ARTICLES_FILEPATH)

/home/nitin.jain/ner_aleju/dataset/sample.txt


In [9]:
from itertools import chain
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
def test_on_mycorpus(args):
    print("Testing on mycorpus (%s)..." % (cfg.ARTICLES_FILEPATH))
    test_on_articles(args, load_articles(cfg.ARTICLES_FILEPATH),
                     nb_append=10000)

def test_on_articles(identifier, articles, nb_append=None):
    print("Loading tagger...")
    tagger = pycrfsuite.Tagger()
    tagger.open(identifier)

    # create feature generators
    # this may take a while
    print("Creating features...")
    feature_generators = features.create_features()

    # create window generator
    print("Loading windows...")
    windows = load_windows(articles, cfg.WINDOW_SIZE, feature_generators, only_labeled_windows=True)

    # load feature lists and label lists (X, Y)
    # this may take a while
    all_feature_values_lists = []
    correct_label_chains = []
    for fvlist, labels in generate_examples(windows, nb_append=nb_append):
        all_feature_values_lists.append(fvlist)
        correct_label_chains.append(labels)

    # generate predicted chains of labels
    print("Testing on %d windows..." % (len(all_feature_values_lists)))
    predicted_label_chains = [tagger.tag(fvlists) for fvlists in all_feature_values_lists]

    # print classification report (precision, recall, f1)
    print(bio_classification_report(correct_label_chains, predicted_label_chains))

def bio_classification_report(y_true, y_pred):
    lbin = LabelBinarizer()
    y_true_combined = lbin.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lbin.transform(list(chain.from_iterable(y_pred)))

    #tagset = set(lbin.classes_) - {NO_NE_LABEL}
    tagset = set(lbin.classes_)
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lbin.classes_)}

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset,
    )
 

In [10]:
test_on_mycorpus("run2_sample45000_withunigram")

Testing on mycorpus (/home/nitin.jain/ner_aleju/dataset/sample.txt)...
Loading tagger...
Creating features...
Loading top N unigrams...
Loading all unigrams...
Loading person name unigrams...
Creating gazetteer...
Loading windows...
Generated 500 of max 10000 examples
Generated 1000 of max 10000 examples
Generated 1500 of max 10000 examples
Generated 2000 of max 10000 examples
Generated 2500 of max 10000 examples
Generated 3000 of max 10000 examples
Generated 3500 of max 10000 examples
Generated 4000 of max 10000 examples
Generated 4500 of max 10000 examples
Generated 5000 of max 10000 examples
Generated 5500 of max 10000 examples
Generated 6000 of max 10000 examples
Generated 6500 of max 10000 examples
Generated 7000 of max 10000 examples
Generated 7500 of max 10000 examples
Generated 8000 of max 10000 examples
Generated 8500 of max 10000 examples
Generated 9000 of max 10000 examples
Generated 9500 of max 10000 examples
Generated 10000 of max 10000 examples
Testing on 10000 windows...

In [2]:
from model.unigrams import Unigrams
print(cfg.UNIGRAMS_FILEPATH)
print("Collecting unigrams...")
ug_all = Unigrams()
ug_all.fill_from_articles(cfg.ARTICLES_FILEPATH, verbose=True)
ug_all.write_to_file(cfg.UNIGRAMS_FILEPATH)
ug_all = None

print("Collecting person names (label=PER)...")
ug_names = Unigrams()
ug_names.fill_from_articles_labels(cfg.ARTICLES_FILEPATH, ["PER"], verbose=True)
ug_names.write_to_file(cfg.UNIGRAMS_PERSON_FILEPATH)
    

/home/nitin.jain/ner_aleju/dataset/preprocessing/unigrams.txt
Article 0
Article 1000
Article 2000
Article 3000
Article 4000
Article 5000
Article 6000
Article 7000
Article 8000
Article 9000
Article 10000
Article 11000
Article 12000
Article 13000
Article 14000
Article 15000
Article 16000
Article 17000
Article 18000
Article 19000
Article 20000
Article 21000
Article 22000
Article 23000
Article 24000
Article 25000
Article 26000
Article 27000
Article 28000
Article 29000
Article 30000
Article 31000
Article 32000
Article 33000
Article 34000
Article 35000
Article 36000
Article 37000
Article 38000
Article 39000
Article 40000
Article 41000
Article 42000
Article 43000
Article 44000
the 49314
. 44816
, 30742
of 24719
in 24715
to 21858
a 19226
and 18683
The 10612
's 10252
for 7992
has 6781
is 6324
on 6291
that 5917
have 5117
with 5058
said 4988
was 4563
says 4356
from 4265
by 4195
at 4058
say 3908
as 3869
U.S. 3864
an 3607
are 3519
" 3412
were 3302
will 3221
his 3093
Mr. 2913
it 2826
government 2826

/home/nitin.jain/ner_aleju/preprocessing/unigrams.txt


In [131]:
ay=dict()
ay['a1']=2
ay['a2']=0
ay['a3']=1
ay
for i,(j,k) in enumerate(ay.items()):
    print(i,j,k)

0 a1 2
1 a2 0
2 a3 1


In [116]:
ax=Counter()
ax.update(['a1','a1','a1','a2','a2','a3'])
x=ax.most_common()
for i,(j,k) in enumerate(x):
    print(i,j,k)

0 a1 3
1 a2 2
2 a3 1


In [16]:
import xgboost as xgb